In [4]:
import pandas as pd
from surprise import Dataset, Reader

import os

DATA_DIR = "data"
train_data = pd.read_csv(os.path.join(DATA_DIR, "train.csv"))
X = pd.read_csv(os.path.join(DATA_DIR, "test.csv"))

reader = Reader(rating_scale=(0, 10))
data = Dataset.load_from_df(train_data, reader)
all_train = data.build_full_trainset()

In [ ]:
# Surprise
from surprise import SVD
from surprise.model_selection import GridSearchCV


# params = {
#     "lam": [0.1, 0.01, 0.001, 0.0001],
#     "K": list(range(5, 51, 10)),
#     "tol": [1e-2, 1e-3, 1e-4, 1e-5],
#     "rd_ratio": list(np.arange(0.1, 1.51, 0.2)),
# }

# search_result = GridSearchCV(
#     SVDpp(n_user, n_item, iterNum=200, verbose=0),
#     param_grid=params,
#     scoring=score,
#     cv=FOLD,
#     n_jobs=-1,
#     verbose=3,
# )

param_grid = {
    "n_epochs": [2500],
    "lr_all": [0.002, 0.005, 0.01, 0.1],
    "reg_all": [0.4, 0.6, 0.8, 0.2],
    "n_factors": [100, 200, 150, 50],
}
gs = GridSearchCV(SVD, param_grid, measures=["rmse"], cv=5, n_jobs=-1, joblib_verbose=3)
gs.fit(data)
print(gs.best_score["rmse"])
print(gs.best_params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:   14.8s
[Parallel(n_jobs=-1)]: Done 104 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 264 tasks      | elapsed:  4.1min


0.7526654930561751
{'rmse': {'n_epochs': 2500, 'lr_all': 0.002, 'reg_all': 0.2, 'n_factors': 50}}


[Parallel(n_jobs=-1)]: Done 320 out of 320 | elapsed:  4.9min finished


In [6]:
from surprise import SVD

svd = SVD(n_factors=200, n_epochs=500, lr_all=0.001, reg_all=0.185, biased=False)
svd.fit(all_train)

In [15]:
import pandas as pd

predictions = pd.DataFrame({}, columns=["Id", "rating"])

for Id, (user_id, item_id) in enumerate(X.values):
    predictions.loc[-1] = [Id, svd.predict(user_id, item_id).est]
    predictions.index = predictions.index + 1
predictions = predictions.set_index("Id").sort_index()
predictions.index = predictions.index.astype(int)
predictions.to_csv("svd.csv")

In [1]:
predictions

NameError: name 'predictions' is not defined